#### GSE243529 組合預測

In [1]:
#GSE243529
import pandas as pd
# liquid_train=pd.read_csv("../../乳癌液態/GSE243529/train/all_beta_normalized_train.csv")
# liquid_test=pd.read_csv("../../乳癌液態/GSE243529/test/all_beta_normalized_test.csv")

In [88]:
input_path = (
    "../result/GDC_breast_tissue_450k_GSE243529/RFE/cluster_RFE_GSE243529_hyper.csv"  # example
)
method_gene = pd.read_csv(input_path)
method_gene

,ID,gene,dbeta,feature,cluster
0,cg23631930,CMTM5,0.172075,TSS200,2
1,cg12440062,CRISP2,0.200273,TSS200,2
2,cg05724197,CX3CL1,0.198136,TSS1500,2
3,cg04225088,HIPK3,0.150001,TSS1500,4
4,cg11775521,KCNE3,0.263576,TSS200,3
5,cg04927004,MIR124-3,0.177844,TSS1500,1
6,cg07211259,PDCD1LG2,0.152846,TSS200,3
7,cg22746058,PTF1A,0.189984,TSS1500,4
8,cg26106778,RING1,0.186264,TSS1500,4
9,cg05884032,SALL3,0.151857,TSS200,4


In [89]:
# liquid_train.set_index('Unnamed: 0', inplace=True)#df1
# liquid_test.set_index('Unnamed: 0', inplace=True)#df2

# liquid = pd.merge(liquid_train, liquid_test, left_index=True, right_index=True, suffixes=('_df1', '_df2'))
# liquid.reset_index(inplace=True)
liquid = liquid_test[liquid_test['Unnamed: 0'].isin(method_gene['ID'])]
liquid = liquid.iloc[:,::2]
liquid

NameError: name 'liquid_test' is not defined

In [10]:
X_train = liquid.iloc[:, 1::]
x = [X_train.iloc[:, i].values.flatten().tolist() for i in range(261)]
# y = [ (1 if i < 128 else 0)  for i in range(262)] # 128 tumor 134 normal
y = [(1 if i < 134 else 0)  for i in range(261)] # 134 tumor 127 normal
# y=liquid_label_train+liquid_label_test

In [49]:
import xgboost as xgb
from sklearn.metrics import roc_auc_score, f1_score,confusion_matrix,accuracy_score,precision_score
from itertools import combinations
from sklearn.model_selection import cross_val_score,cross_val_predict,train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import GridSearchCV, cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix, roc_auc_score, f1_score,matthews_corrcoef
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
import xgboost as xgb
import pandas as pd
import numpy as np
from itertools import combinations


In [90]:
def is_unique(lst):
    return len(lst) == len(set(lst))

In [91]:
param_grids = {
    "SVM": {
        'kernel': ['rbf', 'poly', 'sigmoid'],
        'C':[0.01,0.1,1,10,20,30,50,100,200]
        },
    "Logistic Regression": {
        "C": [0.0001,0.001,0.01,0.1,1,10,20], 
    #     "solver": ["liblinear"]
        },
    "Decision Tree": {
        "max_depth": [1,2, 3, 4],
        "min_samples_split": [10,20,30,40,50,60,70],
        "min_samples_leaf": [10,20,30,40,50,60,70],
        },
    "RandomForest": {
        "n_estimators": [100,200,300,400,500,600],
        "min_samples_split": [60,70,80,90],
        "min_samples_leaf": [20,30,40,50,60,70,80],
    },
    "XGBoost": {
        "learning_rate": [0.001, 0.01, 0.1,1,10],
        "n_estimators": [10,20,30,40,50,100,200,300],
        "max_depth": [0,1,2, 3, 4],
        "subsample": [0.5,0.6, 0.7, 0.8,0.9],
    },
}

In [92]:
#cv=5
target_labels = method_gene['ID']
# 兩個一組
combinations_list = list(combinations(target_labels, 2))
result=[]

models = {
    'XGBoost': xgb.XGBClassifier(),
    'RandomForest': RandomForestClassifier(),
    'SVM': SVC(),
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier()
}

for model_name, model in models.items():
    param_grid = param_grids[model_name]
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
    # 所有可能的組合
    for combination in combinations_list:

        GO_1=int(method_gene.loc[method_gene['ID'] == combination[0]]["Cluster"])
        GO_2=int(method_gene.loc[method_gene['ID'] == combination[1]]["Cluster"])
        # GO_3=int(method_gene.loc[method_gene['ID'] == combination[2]]["Cluster"])

        lst=[GO_1,GO_2]

        if(is_unique(lst)):
            d1_values = liquid.loc[liquid['Unnamed: 0']==combination[0]]
            d2_values = liquid.loc[liquid['Unnamed: 0']==combination[1]]
            # d3_values = liquid.loc[liquid['Unnamed: 0']==combination[2]]
        
            d1 = d1_values.iloc[:, 1::]
            d2 = d2_values.iloc[:, 1::]
            # d3 = d3_values.iloc[:, 1::]

            d1 = d1.values.flatten().tolist()
            d2 = d2.values.flatten().tolist()
            # d3 = d3.values.flatten().tolist()

            x = []

            for i in range(261):
                x.append([d1[i],d2[i]])

            grid_search.fit(x, y)
            model = grid_search.best_estimator_

            cv_scores = cross_val_score(model, x, y, cv=5, scoring='accuracy')
            average_accuracy = cv_scores.mean()

            # 獲取模型預測結果
            y_pred = cross_val_predict(model, x, y, cv=5)
            
            # 預測
            tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()
            sensitivity = tp / (tp + fn)
            specificity = tn / (tn + fp)
            precision = precision_score(y, y_pred)
            auc_score = roc_auc_score(y, y_pred)
            f1 = f1_score(y, y_pred)

            sorted_combination = [x for _, x in sorted(zip(lst, combination))]
            result.append([model_name] + sorted_combination + [average_accuracy, auc_score, f1, sensitivity,specificity,precision])
            
# result

KeyError: 'Cluster'

In [15]:
from sklearn.metrics import classification_report
import numpy as np
print(classification_report(y, y_pred,labels=np.unique(y_pred)))

              precision    recall  f1-score   support

           0       0.58      0.84      0.69       127
           1       0.74      0.43      0.55       134

    accuracy                           0.63       261
   macro avg       0.66      0.64      0.62       261
weighted avg       0.67      0.63      0.62       261



In [16]:
df=pd.DataFrame(result, columns=['Model','ID1', 'ID2', 'average_accuracy','AUC','f1_score','sensitivity','specificity','precision'])
df.to_csv("../result/ml_combination_GSE243529.csv", index=False)

#### GSE89093

In [82]:
import pandas as pd
liquid=pd.read_csv("../../../乳癌液態/GSE89093/GSE89093_all_beta_normalized.csv")
liquid_label= pd.read_csv("../../../乳癌液態/GSE89093/GSE89093_phenotype.csv")
liquid_label=pd.DataFrame(liquid_label["cancer_status"])
liquid= liquid[liquid['ID_REF'].isin(method_gene['ID'])]
liquid

,ID_REF,0,1,2,3,4,5,6,7,8,...,82,83,84,85,86,87,88,89,90,91
14425,cg00727675,0.923720,0.888544,0.933927,0.921822,0.913386,0.918880,0.899692,0.877418,0.926345,...,0.920234,0.897382,0.901499,0.888938,0.883442,0.869808,0.908041,0.823555,0.823321,0.904691
22330,cg01136458,0.629773,0.388929,0.403842,0.448295,0.400347,0.557077,0.578220,0.606832,0.462792,...,0.573352,0.447175,0.288036,0.251962,0.454028,0.680926,0.352915,0.500003,0.415023,0.456636
49048,cg02577745,0.946295,0.872282,0.891887,0.855399,0.915583,0.917339,0.902473,0.877006,0.916174,...,0.880396,0.880720,0.907209,0.885693,0.876727,0.840172,0.884062,0.871134,0.826642,0.903992
79297,cg04225088,0.384941,0.386815,0.352786,0.379399,0.360190,0.400846,0.427248,0.531735,0.391726,...,0.459798,0.428145,0.379420,0.488800,0.532247,0.439673,0.302260,0.494411,0.576925,0.470157
80514,cg04290171,0.929038,0.829157,0.851892,0.558312,0.845068,0.759959,0.760061,0.723293,0.829614,...,0.853953,0.744655,0.775008,0.810484,0.875810,0.736209,0.889226,0.774263,0.839483,0.856503
83090,cg04433322,0.623449,0.660168,0.704213,0.680767,0.689672,0.701087,0.616404,0.647557,0.700837,...,0.717888,0.667355,0.690244,0.682635,0.661378,0.506630,0.720344,0.630918,0.595234,0.699909
91862,cg04927004,0.183700,0.271888,0.253845,0.329069,0.276557,0.263308,0.237192,0.308882,0.267855,...,0.243953,0.231732,0.234402,0.218756,0.244129,0.262375,0.231248,0.248455,0.255094,0.250802
105594,cg05724197,0.881470,0.781956,0.791505,0.827658,0.827168,0.816100,0.797156,0.817910,0.867192,...,0.793264,0.893796,0.859082,0.858964,0.786986,0.787780,0.828101,0.771231,0.747346,0.826932
108227,cg05884032,0.035773,0.128722,0.183718,0.117518,0.129957,0.084905,0.097592,0.100434,0.057997,...,0.123427,0.079221,0.053397,0.055639,0.078975,0.116359,0.081860,0.119291,0.111124,0.128325
115323,cg06314202,0.818404,0.797302,0.855027,0.878752,0.880657,0.864989,0.830959,0.833646,0.882240,...,0.837636,0.823607,0.834792,0.837852,0.820540,0.798673,0.833897,0.793097,0.793138,0.835352


In [83]:
X_train = liquid.iloc[:, 1::]
x = [X_train.iloc[:, i].values.flatten().tolist() for i in range(92)]
y = [0 if liquid_label.iloc[i,0] == 'healthy' else 1 for i in range(liquid_label.shape[0])]

In [84]:
def is_unique(lst):
    return len(lst) == len(set(lst))

In [87]:
#cv=5
target_labels = method_gene['ID']
# 三個一組
combinations_list = list(combinations(target_labels, 4))
result=[]

models = {
    'XGBoost': xgb.XGBClassifier(random_state=42),
    'RandomForest': RandomForestClassifier(random_state=42),
    'SVM': SVC(random_state=42),
    'Logistic Regression': LogisticRegression(random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42)
}

for model_name, model in models.items():
    param_grid = param_grids[model_name]
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
    # 所有可能的組合
    for combination in combinations_list:

        GO_1=int(method_gene.loc[method_gene['ID'] == combination[0]]["cluster"].iloc[0])
        GO_2=int(method_gene.loc[method_gene['ID'] == combination[1]]["cluster"].iloc[0])
        GO_3=int(method_gene.loc[method_gene['ID'] == combination[2]]["cluster"].iloc[0])
        GO_4=int(method_gene.loc[method_gene['ID'] == combination[3]]["cluster"].iloc[0])

        lst=[GO_1,GO_2,GO_3,GO_4]

        if(is_unique(lst)):
            d1_values = liquid.loc[liquid['ID_REF']==combination[0]]
            d2_values = liquid.loc[liquid['ID_REF']==combination[1]]
            d3_values = liquid.loc[liquid['ID_REF']==combination[2]]
            d4_values = liquid.loc[liquid['ID_REF']==combination[3]]
        
            d1 = d1_values.iloc[:, 1::]
            d2 = d2_values.iloc[:, 1::]
            d3 = d3_values.iloc[:, 1::]
            d4 = d4_values.iloc[:, 1::]

            d1 = d1.values.flatten().tolist()
            d2 = d2.values.flatten().tolist()
            d3 = d3.values.flatten().tolist()
            d4 = d4.values.flatten().tolist()

            x = []
            for i in range(92):
                x.append([d1[i],d2[i],d3[i],d4[i]])

            grid_search.fit(x, y)
            model = grid_search.best_estimator_
            # print("Best Parameters:", grid_search.best_params_)
            # print("Best Score:", grid_search.best_score_)

            cv_scores = cross_validate(model, x, y, cv=5, scoring='accuracy',return_train_score=True)
            # print(cv_scores["train_score"])
            # print(cv_scores["test_score"])
            if abs(cv_scores["train_score"].mean()-cv_scores["test_score"].mean())>0.1:
                # print(cv_scores["train_score"].mean())
                # print(cv_scores["test_score"].mean())
                print(f"{model_name} overfitting")
            else:
                average_accuracy = cv_scores["test_score"].mean()

                # 獲取模型預測結果
                y_pred = cross_val_predict(model, x, y, cv=5)
                
                # 預測
                tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()
                sensitivity = tp / (tp + fn)
                specificity = tn / (tn + fp)
                precision = precision_score(y, y_pred)
                # auc_score = roc_auc_score(y, y_pred)
                f1 = f1_score(y, y_pred)
                mcc=matthews_corrcoef(y, y_pred)

                sorted_combination = [x for _, x in sorted(zip(lst, combination))]
                result.append([model_name] + sorted_combination + [
                            round(average_accuracy, 2),
                            round(sensitivity, 2),
                            round(specificity, 2),
                            round(precision, 2),
                            round(f1, 2),
                            round(mcc, 2),
                        ])
            
# result

d:\專題\main_project\.venv\Lib\site-packages\numpy\ma\core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting


KeyboardInterrupt: 

In [59]:
from sklearn.metrics import classification_report
import numpy as np
print(classification_report(y, y_pred,labels=np.unique(y_pred)))

              precision    recall  f1-score   support

           0       0.58      0.61      0.60        46
           1       0.59      0.57      0.58        46

    accuracy                           0.59        92
   macro avg       0.59      0.59      0.59        92
weighted avg       0.59      0.59      0.59        92



In [60]:
df=pd.DataFrame(result, columns=['Model','ID1', 'ID2','ID3','ID4', 'accuracy','sensitivity','specificity','precision','f1_score','mcc'])
df.to_csv("../result/predict_combination_GSE89093_cv.csv", index=False)

In [61]:
df

,Model,ID1,ID2,ID3,ID4,accuracy,sensitivity,specificity,precision,f1_score,mcc
0,RandomForest,cg04927004,cg05724197,cg06314202,cg05884032,0.49,0.39,0.59,0.49,0.43,-0.02
1,SVM,cg04927004,cg05724197,cg06314202,cg05884032,0.60,0.57,0.63,0.60,0.58,0.20
2,Logistic Regression,cg04927004,cg05724197,cg06314202,cg05884032,0.55,0.50,0.61,0.56,0.53,0.11
3,Decision Tree,cg04927004,cg05724197,cg06314202,cg05884032,0.59,0.57,0.61,0.59,0.58,0.17


#### GSE148663烏拉圭

In [93]:
import pandas as pd
liquid=pd.read_csv("../../../乳癌液態/GSE148663烏拉圭/all_beta_normalized_breast_liquid_oversample_random.csv")
liquid= liquid[liquid['Unnamed: 0'].isin(method_gene['ID'])]

liquid

,Unnamed: 0,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,...,1.12,1.13,1.14,1.15,1.16,1.17,1.18,1.19,1.20,1.21
37849,cg12440062,0.678392,0.447820,0.807414,0.679293,0.529423,0.578240,0.679293,0.447820,0.807414,...,0.683746,0.744290,0.747794,0.610422,0.688348,0.691779,0.626707,0.790656,0.816347,0.749898
41642,cg26106778,0.339627,0.321448,0.409579,0.457564,0.298333,0.375744,0.457564,0.321448,0.409579,...,0.276257,0.383036,0.325598,0.229268,0.331586,0.224665,0.368883,0.404174,0.353310,0.197995
60030,cg22746058,0.240843,0.258942,0.294675,0.252646,0.208892,0.283940,0.252646,0.258942,0.294675,...,0.131212,0.270709,0.201969,0.143579,0.218607,0.198090,0.202008,0.220056,0.185828,0.144894
64273,cg11775521,0.340537,0.363007,0.472013,0.580303,0.333575,0.498385,0.580303,0.363007,0.472013,...,0.232615,0.412808,0.401492,0.269061,0.341462,0.267166,0.501855,0.458372,0.436315,0.193128
86704,cg05724197,0.756970,0.812436,0.743354,0.639979,0.817312,0.725989,0.639979,0.812436,0.743354,...,0.744022,0.739070,0.764157,0.824474,0.800564,0.794502,0.745813,0.769968,0.737862,0.834317
97890,cg05884032,0.140366,0.153574,0.101803,0.143567,0.117149,0.167759,0.143567,0.153574,0.101803,...,0.109392,0.168775,0.160552,0.107828,0.127585,0.116110,0.134522,0.142000,0.167934,0.100931
107398,cg04927004,0.329177,0.317637,0.323678,0.419554,0.331304,0.349252,0.419554,0.317637,0.323678,...,0.270353,0.331435,0.309608,0.272571,0.321534,0.309170,0.297952,0.332258,0.323200,0.276633
267524,cg07211259,0.308566,0.274011,0.196633,0.309349,0.379668,0.340288,0.309349,0.274011,0.196633,...,0.460417,0.156566,0.363677,0.357828,0.249886,0.293167,0.284308,0.335693,0.196694,0.286145
291041,cg04225088,0.646110,0.699191,0.820077,0.871858,0.602769,0.831151,0.871858,0.699191,0.820077,...,0.564136,0.690014,0.703451,0.537741,0.641671,0.639747,0.782367,0.762892,0.677641,0.409425
338390,cg23631930,0.866250,0.869477,0.866920,0.829180,0.865487,0.844901,0.829180,0.869477,0.866920,...,0.865420,0.848442,0.874326,0.864665,0.851762,0.868581,0.851638,0.852804,0.848497,0.875360


In [94]:
X_train = liquid.iloc[:, 1::]
x = [X_train.iloc[:, i].values.flatten().tolist() for i in range(44)]
y = [ (0 if i < 22 else 1)  for i in range(44)]

In [95]:
#cv=5
target_labels = method_gene['ID']
# 三個一組
combinations_list = list(combinations(target_labels, 4))
result=[]

models = {
    'XGBoost': xgb.XGBClassifier(random_state=42),
    'RandomForest': RandomForestClassifier(random_state=42),
    'SVM': SVC(random_state=42),
    'Logistic Regression': LogisticRegression(random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42)
}

for model_name, model in models.items():
    param_grid = param_grids[model_name]
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
    # 所有可能的組合
    for combination in combinations_list:

        GO_1=int(method_gene.loc[method_gene['ID'] == combination[0]]["cluster"].iloc[0])
        GO_2=int(method_gene.loc[method_gene['ID'] == combination[1]]["cluster"].iloc[0])
        GO_3=int(method_gene.loc[method_gene['ID'] == combination[2]]["cluster"].iloc[0])
        GO_4=int(method_gene.loc[method_gene['ID'] == combination[3]]["cluster"].iloc[0])

        lst=[GO_1,GO_2,GO_3,GO_4]

        if(is_unique(lst)):
            d1_values = liquid.loc[liquid['Unnamed: 0']==combination[0]]
            d2_values = liquid.loc[liquid['Unnamed: 0']==combination[1]]
            d3_values = liquid.loc[liquid['Unnamed: 0']==combination[2]]
            d4_values = liquid.loc[liquid['Unnamed: 0']==combination[3]]
        
            d1 = d1_values.iloc[:, 1::]
            d2 = d2_values.iloc[:, 1::]
            d3 = d3_values.iloc[:, 1::]
            d4 = d4_values.iloc[:, 1::]

            d1 = d1.values.flatten().tolist()
            d2 = d2.values.flatten().tolist()
            d3 = d3.values.flatten().tolist()
            d4 = d4.values.flatten().tolist()

            x = []

            for i in range(44):
                x.append([d1[i],d2[i],d3[i],d4[i]])

            grid_search.fit(x, y)
            model = grid_search.best_estimator_
            # print("Best Parameters:", grid_search.best_params_)
            # print("Best Score:", grid_search.best_score_)

            cv_scores = cross_validate(model, x, y, cv=5, scoring='accuracy',return_train_score=True)
            # print(cv_scores["train_score"])
            # print(cv_scores["test_score"])
            if abs(cv_scores["train_score"].mean()-cv_scores["test_score"].mean())>0.1:
                # print(cv_scores["train_score"].mean())
                # print(cv_scores["test_score"].mean())
                print(f"{model_name} overfitting")
            else:
                average_accuracy = cv_scores["test_score"].mean()

                # 獲取模型預測結果
                y_pred = cross_val_predict(model, x, y, cv=5)
                
                # 預測
                tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()
                sensitivity = tp / (tp + fn)
                specificity = tn / (tn + fp)
                precision = precision_score(y, y_pred)
                # auc_score = roc_auc_score(y, y_pred)
                f1 = f1_score(y, y_pred)
                mcc=matthews_corrcoef(y, y_pred)

                sorted_combination = [x for _, x in sorted(zip(lst, combination))]
                result.append([model_name] + sorted_combination + [
                            round(average_accuracy, 2),
                            round(sensitivity, 2),
                            round(specificity, 2),
                            round(precision, 2),
                            round(f1, 2),
                            round(mcc, 2),
                        ])
            
            
# result

XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting


d:\專題\main_project\.venv\Lib\site-packages\numpy\ma\core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


SVM overfitting


d:\專題\main_project\.venv\Lib\site-packages\numpy\ma\core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
d:\專題\main_project\.venv\Lib\site-packages\numpy\ma\core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
d:\專題\main_project\.venv\Lib\site-packages\numpy\ma\core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
d:\專題\main_project\.venv\Lib\site-packages\numpy\ma\core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
d:\專題\main_project\.venv\Lib\site-packages\numpy\ma\core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Decision Tree overfitting
Decision Tree overfitting
Decision Tree overfitting
Decision Tree overfitting


...

Decision Tree overfitting


d:\專題\main_project\.venv\Lib\site-packages\numpy\ma\core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
d:\專題\main_project\.venv\Lib\site-packages\numpy\ma\core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Decision Tree overfitting


d:\專題\main_project\.venv\Lib\site-packages\numpy\ma\core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Decision Tree overfitting


In [96]:
from sklearn.metrics import classification_report
import numpy as np
print(classification_report(y,y_pred,labels=np.unique(y_pred)))

              precision    recall  f1-score   support

           0       0.70      0.73      0.71        22
           1       0.71      0.68      0.70        22

    accuracy                           0.70        44
   macro avg       0.70      0.70      0.70        44
weighted avg       0.70      0.70      0.70        44



In [97]:
df=pd.DataFrame(result, columns=['Model','ID1', 'ID2','ID3','ID4', 'accuracy','sensitivity','specificity','precision','f1_score','mcc'])
df.to_csv("../result/ml_combination_GSE148663_cv_hyper.csv", index=False)

In [98]:
df

,Model,ID1,ID2,ID3,ID4,accuracy,sensitivity,specificity,precision,f1_score,mcc
0,XGBoost,cg04927004,cg23631930,cg11775521,cg04225088,0.93,0.86,1.00,1.00,0.93,0.87
1,XGBoost,cg04927004,cg23631930,cg07211259,cg04225088,0.84,0.77,0.91,0.89,0.83,0.69
2,XGBoost,cg04927004,cg23631930,cg04433322,cg04225088,0.86,0.77,0.95,0.94,0.85,0.74
3,XGBoost,cg04927004,cg23631930,cg11775521,cg22746058,0.91,0.91,0.91,0.91,0.91,0.82
4,XGBoost,cg04927004,cg23631930,cg11775521,cg26106778,0.93,0.86,1.00,1.00,0.93,0.87
...,...,...,...,...,...,...,...,...,...,...,...
209,Decision Tree,cg04927004,cg05724197,cg07211259,cg05884032,0.75,0.68,0.82,0.79,0.73,0.50
210,Decision Tree,cg04927004,cg05724197,cg04433322,cg22746058,0.86,0.91,0.82,0.83,0.87,0.73
211,Decision Tree,cg04927004,cg05724197,cg06314202,cg22746058,0.86,0.91,0.82,0.83,0.87,0.73
212,Decision Tree,cg04927004,cg05724197,cg06314202,cg26106778,0.71,0.68,0.73,0.71,0.70,0.41


#### GSE133918母乳

In [70]:
import pandas as pd
liquid=pd.read_csv("../../../乳癌液態/GSE133918_breast_milk/all_beta_normalized_milk.csv")
liquid= liquid[liquid['Unnamed: 0'].isin(method_gene['ID'])]

liquid

,Unnamed: 0,1,2,3,4,5,6,7,8,9,...,151,152,153,154,155,156,157,158,159,160
85332,cg05724197,0.293272,0.293272,0.452109,0.452109,0.372597,0.372597,0.409512,0.409512,0.821222,...,0.412916,0.412916,0.455212,0.455212,0.295880,0.295880,0.545080,0.545080,0.409547,0.409547
96373,cg05884032,0.108651,0.108651,0.077850,0.077850,0.099637,0.099637,0.079464,0.079464,0.048079,...,0.207784,0.207784,0.185010,0.185010,0.186526,0.186526,0.144210,0.144210,0.154008,0.154008
105778,cg04927004,0.156479,0.156479,0.168561,0.168561,0.155670,0.155670,0.172152,0.172152,0.247723,...,0.269992,0.269992,0.155995,0.155995,0.201166,0.201166,0.228621,0.228621,0.204856,0.204856
350849,cg06314202,0.785073,0.785073,0.689295,0.689295,0.733190,0.733190,0.721827,0.721827,0.872115,...,0.602235,0.602235,0.814527,0.814527,0.726077,0.726077,0.801600,0.801600,0.764506,0.764506


In [71]:
X_train = liquid.iloc[:, 1::2]
x = [X_train.iloc[:, i].values.flatten().tolist() for i in range(80)]
y = [ (0 if i < 64 else 1)  for i in range(80)]

In [77]:
#cv=5
target_labels = method_gene['ID']
# 三個一組
combinations_list = list(combinations(target_labels, 4))
result=[]

models = {
    'XGBoost': xgb.XGBClassifier(random_state=42),
    'RandomForest': RandomForestClassifier(random_state=42),
    'SVM': SVC(random_state=42),
    'Logistic Regression': LogisticRegression(random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42)
}

for model_name, model in models.items():
    param_grid = param_grids[model_name]
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
    # 所有可能的組合
    for combination in combinations_list:

        GO_1=int(method_gene.loc[method_gene['ID'] == combination[0]]["cluster"].iloc[0])
        GO_2=int(method_gene.loc[method_gene['ID'] == combination[1]]["cluster"].iloc[0])
        GO_3=int(method_gene.loc[method_gene['ID'] == combination[2]]["cluster"].iloc[0])
        GO_4=int(method_gene.loc[method_gene['ID'] == combination[3]]["cluster"].iloc[0])
        

        lst=[GO_1,GO_2,GO_3,GO_4]

        if(is_unique(lst)):
            d1_values = liquid.loc[liquid['Unnamed: 0']==combination[0]]
            d2_values = liquid.loc[liquid['Unnamed: 0']==combination[1]]
            d3_values = liquid.loc[liquid['Unnamed: 0']==combination[2]]
            d4_values = liquid.loc[liquid['Unnamed: 0']==combination[3]]
           
        
            d1 = d1_values.iloc[:, 1::]
            d2 = d2_values.iloc[:, 1::]
            d3 = d3_values.iloc[:, 1::]
            d4 = d4_values.iloc[:, 1::]
           

            d1 = d1.values.flatten().tolist()
            d2 = d2.values.flatten().tolist()
            d3 = d3.values.flatten().tolist()
            d4 = d4.values.flatten().tolist()
           

            x = []

            for i in range(80):
                x.append([d1[i],d2[i],d3[i],d4[i]])

            grid_search.fit(x, y)
            model = grid_search.best_estimator_
            print("Best Parameters:", grid_search.best_params_)
            print("Best Score:", grid_search.best_score_)

            cv_scores = cross_validate(model, x, y, cv=5, scoring='accuracy',return_train_score=True)
            print(cv_scores["train_score"])
            print(cv_scores["test_score"])
            if abs(cv_scores["train_score"].mean()-cv_scores["test_score"].mean())>0.1:
                print(cv_scores["train_score"].mean())
                print(cv_scores["test_score"].mean())
                print(f"{model_name} overfitting")
            else:
                average_accuracy = cv_scores["test_score"].mean()

                # 獲取模型預測結果
                y_pred = cross_val_predict(model, x, y, cv=5)
                
                # 預測
                tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()
                sensitivity = tp / (tp + fn)
                specificity = tn / (tn + fp)
                precision = precision_score(y, y_pred)
                # auc_score = roc_auc_score(y, y_pred)
                f1 = f1_score(y, y_pred)
                mcc=matthews_corrcoef(y, y_pred)

                sorted_combination = [x for _, x in sorted(zip(lst, combination))]
                result.append([model_name] + sorted_combination + [
                            round(average_accuracy, 2),
                            round(sensitivity, 2),
                            round(specificity, 2),
                            round(precision, 2),
                            round(f1, 2),
                            round(mcc, 2),
                        ])
            
# result

Best Parameters: {'learning_rate': 0.1, 'max_depth': 0, 'n_estimators': 50, 'subsample': 0.8}
Best Score: 0.825
[1.      1.      1.      1.      0.96875]
[0.875 0.875 0.875 0.625 0.875]
0.99375
0.825
XGBoost overfitting


d:\專題\main_project\.venv\Lib\site-packages\numpy\ma\core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best Parameters: {'min_samples_leaf': 20, 'min_samples_split': 60, 'n_estimators': 100}
Best Score: 0.8
[0.796875 0.796875 0.796875 0.796875 0.8125  ]
[0.8125 0.8125 0.8125 0.8125 0.75  ]


d:\專題\main_project\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Best Parameters: {'C': 0.01, 'kernel': 'rbf'}
Best Score: 0.8
[0.796875 0.796875 0.796875 0.796875 0.8125  ]
[0.8125 0.8125 0.8125 0.8125 0.75  ]
Best Parameters: {'C': 0.0001}
Best Score: 0.8
[0.796875 0.796875 0.796875 0.796875 0.8125  ]
[0.8125 0.8125 0.8125 0.8125 0.75  ]


d:\專題\main_project\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\專題\main_project\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Best Parameters: {'max_depth': 1, 'min_samples_leaf': 10, 'min_samples_split': 10}
Best Score: 0.8
[0.796875 0.796875 0.796875 0.796875 0.8125  ]
[0.8125 0.8125 0.8125 0.8125 0.75  ]


d:\專題\main_project\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [78]:
from sklearn.metrics import classification_report
import numpy as np
print(classification_report(y,y_pred,labels=np.unique(y_pred)))

              precision    recall  f1-score   support

           0       0.80      1.00      0.89        64

   micro avg       0.80      1.00      0.89        64
   macro avg       0.80      1.00      0.89        64
weighted avg       0.80      1.00      0.89        64



In [75]:
df=pd.DataFrame(result, columns=['Model','ID1', 'ID2','ID3','ID4', 'accuracy','sensitivity','specificity','precision','f1_score','mcc'])
df.to_csv("../result/ml_combination_GSE133918.csv", index=False)

In [79]:
df

,Model,ID1,ID2,ID3,ID4,accuracy,sensitivity,specificity,precision,f1_score,mcc
0,RandomForest,cg04927004,cg05724197,cg06314202,cg05884032,0.8,0.0,1.0,0.0,0.0,0.0
1,SVM,cg04927004,cg05724197,cg06314202,cg05884032,0.8,0.0,1.0,0.0,0.0,0.0
2,Logistic Regression,cg04927004,cg05724197,cg06314202,cg05884032,0.8,0.0,1.0,0.0,0.0,0.0
3,Decision Tree,cg04927004,cg05724197,cg06314202,cg05884032,0.8,0.0,1.0,0.0,0.0,0.0


#### 850K組織

In [99]:
import pandas as pd
tissue=pd.read_csv("../../../乳癌資料集850/all_beta_normalized_850(GSE237036_RAW).csv")
tissue= tissue[tissue['Unnamed: 0'].isin(method_gene['ID'])]
tissue

,Unnamed: 0,1,2,3,4,5,6,7,8,9,...,151,152,153,154,155,156,157,158,159,160
17276,cg04225088,0.660342,0.660342,0.857121,0.857121,0.738259,0.738259,0.634618,0.634618,0.579239,...,0.782463,0.782463,0.761348,0.761348,0.643839,0.643839,0.747830,0.747830,0.667708,0.667708
164214,cg06314202,0.728585,0.728585,0.694682,0.694682,0.760598,0.760598,0.813221,0.813221,0.790985,...,0.745460,0.745460,0.730565,0.730565,0.747791,0.747791,0.786396,0.786396,0.755846,0.755846
169957,cg23631930,0.828017,0.828017,0.731314,0.731314,0.783682,0.783682,0.856924,0.856924,0.891339,...,0.810462,0.810462,0.772989,0.772989,0.766709,0.766709,0.824339,0.824339,0.797026,0.797026
223445,cg07211259,0.230981,0.230981,0.173913,0.173913,0.178107,0.178107,0.255519,0.255519,0.163001,...,0.272251,0.272251,0.197166,0.197166,0.171586,0.171586,0.271625,0.271625,0.193946,0.193946
297671,cg04433322,0.632727,0.632727,0.449648,0.449648,0.542075,0.542075,0.670599,0.670599,0.661606,...,0.548861,0.548861,0.598534,0.598534,0.624525,0.624525,0.626541,0.626541,0.610374,0.610374
656375,cg05884032,0.126258,0.126258,0.248068,0.248068,0.188227,0.188227,0.113317,0.113317,0.179600,...,0.187490,0.187490,0.297772,0.297772,0.271526,0.271526,0.263881,0.263881,0.162370,0.162370
657308,cg26106778,0.249123,0.249123,0.366003,0.366003,0.337686,0.337686,0.206491,0.206491,0.224944,...,0.224654,0.224654,0.389349,0.389349,0.310641,0.310641,0.220612,0.220612,0.329889,0.329889
665077,cg05724197,0.843893,0.843893,0.756403,0.756403,0.802522,0.802522,0.877246,0.877246,0.903469,...,0.815318,0.815318,0.786951,0.786951,0.804395,0.804395,0.773011,0.773011,0.775457,0.775457
666900,cg11775521,0.434653,0.434653,0.578097,0.578097,0.504674,0.504674,0.338815,0.338815,0.381638,...,0.485076,0.485076,0.468729,0.468729,0.458505,0.458505,0.452506,0.452506,0.459169,0.459169
682822,cg12440062,0.670549,0.670549,0.717885,0.717885,0.748806,0.748806,0.622496,0.622496,0.863139,...,0.585302,0.585302,0.553144,0.553144,0.760399,0.760399,0.707429,0.707429,0.701104,0.701104


In [100]:
X_train = tissue.iloc[:, 1::2]
y = [ (1 if i < 50 else 0)  for i in range(80)]

In [101]:
from sklearn.model_selection import StratifiedKFold,cross_validate
from sklearn.metrics import accuracy_score


In [103]:
#cv=5
target_labels = method_gene['ID']
# 三個一組
combinations_list = list(combinations(target_labels, 4))
result=[]

models = {
    'XGBoost': xgb.XGBClassifier(random_state=42),
    'RandomForest': RandomForestClassifier(random_state=42),
    'SVM': SVC(random_state=42),
    'Logistic Regression': LogisticRegression(random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42)
}

for model_name, model in models.items():
    param_grid = param_grids[model_name]
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
    # 所有可能的組合
    for combination in combinations_list:

        GO_1=int(method_gene.loc[method_gene['ID'] == combination[0]]["cluster"].iloc[0])
        GO_2=int(method_gene.loc[method_gene['ID'] == combination[1]]["cluster"].iloc[0])
        GO_3=int(method_gene.loc[method_gene['ID'] == combination[2]]["cluster"].iloc[0])
        GO_4=int(method_gene.loc[method_gene['ID'] == combination[3]]["cluster"].iloc[0])


        lst=[GO_1,GO_2,GO_3,GO_4]

        if(is_unique(lst)):
            d1_values = tissue.loc[tissue['Unnamed: 0']==combination[0]]
            d2_values = tissue.loc[tissue['Unnamed: 0']==combination[1]]
            d3_values = tissue.loc[tissue['Unnamed: 0']==combination[2]]
            d4_values = tissue.loc[tissue['Unnamed: 0']==combination[3]]
        
        
            d1 = d1_values.iloc[:, 1::]
            d2 = d2_values.iloc[:, 1::]
            d3 = d3_values.iloc[:, 1::]
            d4 = d4_values.iloc[:, 1::]
           

            d1 = d1.values.flatten().tolist()
            d2 = d2.values.flatten().tolist()
            d3 = d3.values.flatten().tolist()
            d4 = d4.values.flatten().tolist()
          
            x = []

            for i in range(80):
                x.append([d1[i],d2[i],d3[i],d4[i]])
            
            grid_search.fit(x, y)
            model = grid_search.best_estimator_
            # print("Best Parameters:", grid_search.best_params_)
            # print("Best Score:", grid_search.best_score_)

            cv_scores = cross_validate(model, x, y, cv=5, scoring='accuracy',return_train_score=True)
            # print(cv_scores["train_score"])
            # print(cv_scores["test_score"])
            if abs(cv_scores["train_score"].mean()-cv_scores["test_score"].mean())>0.1:
                # print(cv_scores["train_score"].mean())
                # print(cv_scores["test_score"].mean())
                print(f"{model_name} overfitting")
            else:
                average_accuracy = cv_scores["test_score"].mean()

                # 獲取模型預測結果
                y_pred = cross_val_predict(model, x, y, cv=5)
                
                # 預測
                tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()
                sensitivity = tp / (tp + fn)
                specificity = tn / (tn + fp)
                precision = precision_score(y, y_pred)
                # auc_score = roc_auc_score(y, y_pred)
                f1 = f1_score(y, y_pred)
                mcc=matthews_corrcoef(y, y_pred)

                sorted_combination = [x for _, x in sorted(zip(lst, combination))]
                result.append([model_name] + sorted_combination + [
                            round(average_accuracy, 2),
                            round(sensitivity, 2),
                            round(specificity, 2),
                            round(precision, 2),
                            round(f1, 2),
                            round(mcc, 2),
                        ])
            
# result

XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting
XGBoost overfitting


d:\專題\main_project\.venv\Lib\site-packages\numpy\ma\core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


SVM overfitting
SVM overfitting
SVM overfitting
SVM overfitting
SVM overfitting
SVM overfitting
SVM overfitting


d:\專題\main_project\.venv\Lib\site-packages\numpy\ma\core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
d:\專題\main_project\.venv\Lib\site-packages\numpy\ma\core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
d:\專題\main_project\.venv\Lib\site-packages\numpy\ma\core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Decision Tree overfitting
Decision Tree overfitting
Decision Tree overfitting


...

Decision Tree overfitting


d:\專題\main_project\.venv\Lib\site-packages\numpy\ma\core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Decision Tree overfitting


d:\專題\main_project\.venv\Lib\site-packages\numpy\ma\core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Decision Tree overfitting
Decision Tree overfitting


In [104]:
from sklearn.metrics import classification_report
import numpy as np
print(classification_report(y,y_pred,labels=np.unique(y_pred)))

              precision    recall  f1-score   support

           0       0.62      0.53      0.57        30
           1       0.74      0.80      0.77        50

    accuracy                           0.70        80
   macro avg       0.68      0.67      0.67        80
weighted avg       0.69      0.70      0.70        80



In [105]:
df=pd.DataFrame(result, columns=['Model','ID1', 'ID2','ID3','ID4', 'accuracy','sensitivity','specificity','precision','f1_score','mcc'])
df.to_csv("../result/ml_combination_850K_cv_hyper.csv", index=False)

In [106]:
df

,Model,ID1,ID2,ID3,ID4,accuracy,sensitivity,specificity,precision,f1_score,mcc
0,XGBoost,cg04927004,cg23631930,cg07211259,cg22746058,0.75,0.84,0.60,0.78,0.81,0.45
1,XGBoost,cg04927004,cg12440062,cg11775521,cg04225088,0.72,0.84,0.53,0.75,0.79,0.39
2,XGBoost,cg04927004,cg12440062,cg07211259,cg04225088,0.62,1.00,0.00,0.62,0.77,0.00
3,XGBoost,cg04927004,cg12440062,cg06314202,cg04225088,0.65,0.84,0.33,0.68,0.75,0.20
4,XGBoost,cg04927004,cg12440062,cg11775521,cg22746058,0.68,0.96,0.20,0.67,0.79,0.26
...,...,...,...,...,...,...,...,...,...,...,...
190,Decision Tree,cg04927004,cg05724197,cg07211259,cg05884032,0.70,0.80,0.53,0.74,0.77,0.34
191,Decision Tree,cg04927004,cg05724197,cg04433322,cg22746058,0.62,1.00,0.00,0.62,0.77,0.00
192,Decision Tree,cg04927004,cg05724197,cg04433322,cg26106778,0.62,1.00,0.00,0.62,0.77,0.00
193,Decision Tree,cg04927004,cg05724197,cg04433322,cg05884032,0.70,0.80,0.53,0.74,0.77,0.34


: 

#### 450K組織

In [32]:
tissue_train = pd.read_csv('../../../all_beta_normalized_train.csv')
tissue_test = pd.read_csv('../../../all_beta_normalized_test.csv')

In [33]:
tissue_train.set_index('Unnamed: 0', inplace=True)#df1
tissue_test.set_index('Unnamed: 0', inplace=True)#df2

tissue = pd.merge(tissue_train, tissue_test, left_index=True, right_index=True, suffixes=('_df1', '_df2'))
tissue.reset_index(inplace=True)
tissue = tissue[tissue['Unnamed: 0'].isin(method_gene['ID'])]
tissue = tissue.iloc[:,::2]
tissue

,Unnamed: 0,2_df1,4_df1,6_df1,8_df1,10_df1,12_df1,14_df1,16_df1,18_df1,...,882_df2,884_df2,886_df2,888_df2,890_df2,892,894,896,898,900
41948,cg25707994,0.123813,0.139064,0.150651,0.142857,0.139549,0.146872,0.170277,0.154333,0.142494,...,0.103208,0.093875,0.124883,0.087826,0.104993,0.128541,0.105823,0.116690,0.089958,0.117519
98070,cg24911721,0.819479,0.854576,0.853194,0.895712,0.879786,0.851553,0.908524,0.898320,0.839139,...,0.953194,0.946846,0.955103,0.934092,0.908227,0.950721,0.910843,0.901216,0.934429,0.894763
196397,cg02355304,0.953460,0.966780,0.944214,0.954833,0.973809,0.970132,0.956842,0.954721,0.960031,...,0.920400,0.970129,0.961543,0.940326,0.805315,0.932736,0.968104,0.974910,0.963761,0.660272
235591,cg19935065,0.672036,0.734419,0.752292,0.712272,0.724383,0.716385,0.749410,0.682247,0.713020,...,0.842407,0.650698,0.157748,0.807882,0.559512,0.269160,0.660809,0.770899,0.839418,0.689251


In [34]:
X_train = tissue.iloc[:, 1::]
x = [X_train.iloc[:, i].values.flatten().tolist() for i in range(895)]
tissue_label_train = [ (0 if i < 47 else 1)  for i in range(445)] # 398 tumor/47 normal =445
tissue_label_test = [(0 if i < 50 else 1)  for i in range(450)] # 400 tumor/50 normal =450
y=tissue_label_train+tissue_label_test

In [38]:
#cv=5
target_labels = method_gene['ID']
# 三個一組
combinations_list = list(combinations(target_labels, 4))
result=[]

models = {
    'XGBoost': xgb.XGBClassifier(),
    'RandomForest': RandomForestClassifier(),
    'SVM': SVC(),
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier()
}

for model_name, model in models.items():
    param_grid = param_grids[model_name]
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
    # 所有可能的組合
    for combination in combinations_list:

        GO_1=int(method_gene.loc[method_gene['ID'] == combination[0]]["Cluster"])
        GO_2=int(method_gene.loc[method_gene['ID'] == combination[1]]["Cluster"])
        GO_3=int(method_gene.loc[method_gene['ID'] == combination[2]]["Cluster"])
        GO_4=int(method_gene.loc[method_gene['ID'] == combination[3]]["Cluster"])
   

        lst=[GO_1,GO_2,GO_3,GO_4]

        if(is_unique(lst)):
            d1_values = tissue.loc[tissue['Unnamed: 0']==combination[0]]
            d2_values = tissue.loc[tissue['Unnamed: 0']==combination[1]]
            d3_values = tissue.loc[tissue['Unnamed: 0']==combination[2]]
            d4_values = tissue.loc[tissue['Unnamed: 0']==combination[3]]

        
            d1 = d1_values.iloc[:, 1::]
            d2 = d2_values.iloc[:, 1::]
            d3 = d3_values.iloc[:, 1::]
            d4 = d4_values.iloc[:, 1::]


            d1 = d1.values.flatten().tolist()
            d2 = d2.values.flatten().tolist()
            d3 = d3.values.flatten().tolist()
            d4 = d4.values.flatten().tolist()


            x = []
            for i in range(895):
                x.append([d1[i],d2[i],d3[i],d4[i]])

            grid_search.fit(x, y)
            model = grid_search.best_estimator_

            cv_scores = cross_val_score(model, x, y, cv=5, scoring='accuracy')
            average_accuracy = cv_scores.mean()

            # 獲取模型預測結果
            y_pred = cross_val_predict(model, x, y, cv=5)
            
            # 預測
            tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()
            sensitivity = tp / (tp + fn)
            specificity = tn / (tn + fp)
            precision = precision_score(y, y_pred)
            auc_score = roc_auc_score(y, y_pred)
            f1 = f1_score(y, y_pred)

            sorted_combination = [x for _, x in sorted(zip(lst, combination))]
            result.append([model_name] + sorted_combination + [average_accuracy, auc_score, f1, sensitivity,specificity,precision])
            
# result

C:\Users\Administrator\AppData\Local\Temp\ipykernel_21096\3320616365.py:21: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  GO_1=int(method_gene.loc[method_gene['ID'] == combination[0]]["Cluster"])
C:\Users\Administrator\AppData\Local\Temp\ipykernel_21096\3320616365.py:22: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  GO_2=int(method_gene.loc[method_gene['ID'] == combination[1]]["Cluster"])
C:\Users\Administrator\AppData\Local\Temp\ipykernel_21096\3320616365.py:23: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  GO_3=int(method_gene.loc[method_gene['ID'] == combination[2]]["Cluster"])
C:\Users\Administrator\AppData\Local\Temp\ipykernel_21096\3320616365.py:24: FutureWarning: Calling int on a single element Seri

In [39]:
from sklearn.metrics import classification_report
import numpy as np
print(classification_report(y, y_pred,labels=np.unique(y_pred)))

              precision    recall  f1-score   support

           0       0.51      0.52      0.51        97
           1       0.94      0.94      0.94       798

    accuracy                           0.89       895
   macro avg       0.72      0.73      0.72       895
weighted avg       0.89      0.89      0.89       895



In [40]:
df=pd.DataFrame(result, columns=['Model','ID1', 'ID2','ID3','ID4', 'average_accuracy','AUC','f1_score','sensitivity','specificity','precision'])
df.to_csv("../result/ml_combination_450K_aba.csv", index=False)